In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
import statsmodels.api as sm
import scipy.stats as sci
from statsmodels.regression.linear_model import OLS

pd.set_option('float_format', '{:6.3f}'.format)
sns.set(style='ticks')

import warnings
warnings.filterwarnings("ignore")

In [2]:
from pylab import rcParams
%matplotlib inline
rcParams['figure.figsize'] = 20,4

In [3]:
df = pd.read_csv('cape.csv',sep = ";")[['Date','P','E','CAPE']][13:].reset_index()

In [4]:
del df['index']

In [5]:
df['E/P'] = df['E']/df['P']

In [6]:
df_c = pd.read_csv('CAT.csv')[['Date','Adj Close']]

In [7]:
df_c['return'] = df_c['Adj Close'].diff(1)/df_c['Adj Close'].shift(1)

In [8]:
df_r = df_c[['Date','return']][2:].reset_index()
del df_r['index']

In [9]:
df_r.head()

,Date,return
0,1963-02-01,-0.085
1,1963-03-01,0.043
2,1963-04-01,0.021
3,1963-05-01,0.201
4,1963-06-01,-0.028


In [10]:
df.head()

,Date,P,E,CAPE,E/P
0,"1963,01",65.060,3.683,19.259,0.057
1,"1963,02",65.920,3.697,19.469,0.056
2,"1963,03",65.670,3.710,19.288,0.056
3,"1963,04",68.760,3.753,20.150,0.055
4,"1963,05",70.140,3.797,20.508,0.054


In [11]:
df.tail()

,Date,P,E,CAPE,E/P
677,"2019,06",2890.170,135.270,29.284,0.047
678,"2019,07",2996.114,134.480,29.987,0.045
679,"2019,08",2897.498,133.690,28.705,0.046
680,"2019,09",2982.156,132.900,29.230,0.045
681,"2019,1",2977.680,135.407,28.841,0.045


In [12]:
df_r = df_r.reset_index()[['Date','return']]

In [16]:
#1 first model
pd_est_model=pd.DataFrame(columns=['date','error_sq'])
for i in range(0,646):
    model = OLS(df_r.iloc[0+i:36+i,1], sm.add_constant(df.iloc[0+i:36+i,4])['const'])
    result = model.fit(cov_type='HC0')
    oos=(result.params[0] - df_r.iloc[36+i,1])**2
    pd_est_model=pd_est_model.append({'date':df_r.iloc[36+i,0],'error_sq':oos},ignore_index = True)
    print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                 return   R-squared:                      -0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Wed, 04 Mar 2020   Prob (F-statistic):                nan
Time:                        21:28:18   Log-Likelihood:                 48.651
No. Observations:                  36   AIC:                            -95.30
Df Residuals:                      35   BIC:                            -93.72
Df Model:                           0                                         
Covariance Type:                  HC0                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0305      0.010      2.919      0.0

In [17]:
# second model
pd_est_model2=pd.DataFrame(columns=['date','error_sq'])
for i in range(0,646):
    model = OLS(df_r.iloc[0+i:36+i,1], sm.add_constant(df.iloc[0+i:36+i,4]))
    result = model.fit(cov_type='HC0')
    oos=(result.params[0]+result.params[1]*df.iloc[36+i,4] - df_r.iloc[36+i,1])**2
    pd_est_model2=pd_est_model2.append({'date':df_r.iloc[36+i,0],'error_sq':oos},ignore_index = True)
    print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                 return   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                 -0.013
Method:                 Least Squares   F-statistic:                    0.7077
Date:                Wed, 04 Mar 2020   Prob (F-statistic):              0.406
Time:                        21:28:32   Log-Likelihood:                 48.950
No. Observations:                  36   AIC:                            -93.90
Df Residuals:                      34   BIC:                            -90.73
Df Model:                           1                                         
Covariance Type:                  HC0                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3506      0.382      0.919      0.3

In [141]:
pd_est_model.mean()

error_sq    0.008
dtype: float64

In [142]:
pd_est_model2.mean()

error_sq    0.008
dtype: float64

In [143]:
from scipy.stats import ttest_ind
statistic, pvalue = ttest_ind(a=pd_est_model['error_sq'],b=pd_est_model2['error_sq'])
statistic

-0.698491656684882

In [144]:
pd_est_model['error_sq'].sum()

4.870570012070047

In [145]:
pd_est_model2['error_sq'].sum()

5.265146004792021

Actually it is not right that adding an extra regressor will increase the forecast accuracy. The first model is better when we assume that the best prediction is the average return during the past particular period.

In [147]:
#2
df_annual = df[::12].reset_index()
del df_annual['index']

In [185]:
df_annual.head()

,Date,P,E,CAPE,E/P
0,"1963,01",65.060,3.683,19.259,0.057
1,"1964,01",76.450,4.073,21.627,0.053
2,"1965,01",86.120,4.593,23.269,0.053
3,"1966,01",93.320,5.240,24.058,0.056
4,"1967,01",84.450,5.517,20.432,0.065


In [181]:
df_c_annual = df_c[['Date','Adj Close']].shift(-1)[::12]

In [182]:
df_c_annual['return'] = df_c_annual['Adj Close'].diff(1)/df_c_annual['Adj Close'].shift(1)

In [183]:
df_c_annual = df_c_annual[1:].reset_index()
del df_c_annual['index']

In [184]:
df_c_annual.head()

,Date,Adj Close,return
0,1964-01-01,0.197,0.399
1,1965-01-01,0.313,0.588
2,1966-01-01,0.389,0.241
3,1967-01-01,0.325,-0.164
4,1968-01-01,0.331,0.018


In [214]:
# first model
pd_est_model_2=pd.DataFrame(columns=['date','error_sq'])
for i in range(0,37):
    model = OLS(df_c_annual.iloc[0+i:20+i,2], sm.add_constant(df_annual.iloc[0+i:20+i,4])['const'])
    result = model.fit(cov_type='HC0')
    oos=(result.params[0] - df_c_annual.iloc[20+i,2])**2
    pd_est_model_2=pd_est_model_2.append({'date':df_c_annual.iloc[20+i,0],'error_sq':oos},ignore_index = True)

In [215]:
# second model
pd_est_model2_2=pd.DataFrame(columns=['date','error_sq'])
for i in range(0,37):
    model = OLS(df_c_annual.iloc[0+i:20+i,2], sm.add_constant(df_annual.iloc[0+i:20+i,4]))
    result = model.fit(cov_type='HC0')
    oos=(result.params[0]+result.params[1]*df_annual.iloc[20+i,4] - df_c_annual.iloc[20+i,2])**2
    pd_est_model2_2=pd_est_model2_2.append({'date':df_c_annual.iloc[20+i,0],'error_sq':oos},ignore_index = True)

In [216]:
pd_est_model_2['error_sq'].sum()

4.824037823561425

In [217]:
pd_est_model2_2['error_sq'].sum()

4.8042612841011945

The result is a little bit better. I mean the sum of squares is less in the second model but not essentially. They both are still practicularly equal. If I decrease the number of rollings the result will be different, meaning the first model is better according to the sum of squares, but there is still not much difference between these two models. The results are practically the same as in the first trial. It means that there are no needs to add an additional regressor because the regressor 'e/p' does not have any power to predict the future returns whatever the period is. In addition, stock returns are really hard to predict.

In [258]:
#3
df_sp=pd.read_csv('df_sp.csv',sep = ";")

In [259]:
df_sp['E/P']=df_sp['E']/df_sp['P']
df_sp=df_sp[::12]

In [260]:
df_sp['return']= df_sp['P'].diff(1)/df_sp['P'].shift(1)
df_sp_r = df_sp[['Date','return']].shift(-1).reset_index()
del df_sp_r['index']
df_sp_r.head()

,Date,return
0,"1882,01",-0.044
1,"1883,01",-0.019
2,"1884,01",-0.108
3,"1885,01",-0.181
4,"1886,01",0.226


In [261]:
df_sp=df_sp.reset_index()
del df_sp['index']

In [262]:
df_sp.head()

,Date,P,D,E,CAPE,E/P,return
0,"1881,01",6.190,0.265,0.486,18.474,0.078,nan
1,"1882,01",5.920,0.320,0.439,15.679,0.074,-0.044
2,"1883,01",5.810,0.321,0.427,15.270,0.074,-0.019
3,"1884,01",5.180,0.328,0.393,14.433,0.076,-0.108
4,"1885,01",4.240,0.304,0.307,13.130,0.072,-0.181


In [269]:
#10 years

#first model
pd_est_model_3_10=pd.DataFrame(columns=['date','error_sq'])
for i in range(0,128):
    model = OLS(df_sp_r.iloc[0+i:10+i,1], sm.add_constant(df_sp.iloc[0+i:10+i,4])['const'])
    result = model.fit(cov_type='HC0')
    oos=(result.params[0] - df_sp_r.iloc[10+i,1])**2
    pd_est_model_3_10=pd_est_model_3_10.append({'date':df_sp_r.iloc[10+i,0],'error_sq':oos},ignore_index = True)

In [271]:
# second model
pd_est_model2_3_10=pd.DataFrame(columns=['date','error_sq'])
for i in range(0,128):
    model = OLS(df_sp_r.iloc[0+i:10+i,1], sm.add_constant(df_sp.iloc[0+i:10+i,5]))
    result = model.fit(cov_type='HC0')
    oos=(result.params[0]+result.params[1]*df_sp.iloc[10+i,5] - df_sp_r.iloc[10+i,1])**2
    pd_est_model2_3_10=pd_est_model2_3_10.append({'date':df_sp_r.iloc[10+i,0],'error_sq':oos},ignore_index = True)

In [274]:
pd_est_model_3_10['error_sq'].sum()

4.42049854291729

In [275]:
pd_est_model2_3_10['error_sq'].sum()

5.802794941652946

In [276]:
#20 years

#first model 
pd_est_model_3_20=pd.DataFrame(columns=['date','error_sq'])
for i in range(0,118):
    model = OLS(df_sp_r.iloc[0+i:20+i,1], sm.add_constant(df_sp.iloc[0+i:20+i,4])['const'])
    result = model.fit(cov_type='HC0')
    oos=(result.params[0] - df_sp_r.iloc[20+i,1])**2
    pd_est_model_3_20=pd_est_model_3_20.append({'date':df_sp_r.iloc[20+i,0],'error_sq':oos},ignore_index = True)

In [279]:
# second model
pd_est_model2_3_20=pd.DataFrame(columns=['date','error_sq'])
for i in range(0,118):
    model = OLS(df_sp_r.iloc[0+i:20+i,1], sm.add_constant(df_sp.iloc[0+i:20+i,5]))
    result = model.fit(cov_type='HC0')
    oos=(result.params[0]+result.params[1]*df_sp.iloc[20+i,5] - df_sp_r.iloc[20+i,1])**2
    pd_est_model2_3_20=pd_est_model2_3_20.append({'date':df_sp_r.iloc[20+i,0],'error_sq':oos},ignore_index = True)

In [281]:
pd_est_model_3_20['error_sq'].sum()

4.132557567404263

In [282]:
pd_est_model2_3_20['error_sq'].sum()

4.4702309813638115

In [283]:
#30 years 

#first model
pd_est_model_3_30=pd.DataFrame(columns=['date','error_sq'])
for i in range(0,108):
    model = OLS(df_sp_r.iloc[0+i:30+i,1], sm.add_constant(df_sp.iloc[0+i:30+i,4])['const'])
    result = model.fit(cov_type='HC0')
    oos=(result.params[0] - df_sp_r.iloc[30+i,1])**2
    pd_est_model_3_30=pd_est_model_3_30.append({'date':df_sp_r.iloc[30+i,0],'error_sq':oos},ignore_index = True)


In [284]:
#second model
pd_est_model2_3_30=pd.DataFrame(columns=['date','error_sq'])
for i in range(0,108):
    model = OLS(df_sp_r.iloc[0+i:30+i,1], sm.add_constant(df_sp.iloc[0+i:30+i,5]))
    result = model.fit(cov_type='HC0')
    oos=(result.params[0]+result.params[1]*df_sp.iloc[30+i,5] - df_sp_r.iloc[30+i,1])**2
    pd_est_model2_3_30=pd_est_model2_3_30.append({'date':df_sp_r.iloc[30+i,0],'error_sq':oos},ignore_index = True)

In [285]:
pd_est_model_3_30['error_sq'].sum()

3.652791903163243

In [286]:
pd_est_model2_3_30['error_sq'].sum()

3.8160821867940617

In each case the first model is better. There is no much difference with the second trial because there we predict the future return of the stock but here we predict the return of the index. The return of the index is also hard to forecast, that is why the first model shows better predictions as in the second trial (2) because the regrossor 'e/p' does not explain altering in the return. The best prediction is still average past returns. When we use 10 years in sample the result in the second model for forecasting is not so good because we have a few returns and the regression does not work perfectly for predicting the result which is out of sample.

In [287]:
#4
#10 years

#first model
pd_est_model_4_10=pd.DataFrame(columns=['date','error_sq'])
for i in range(0,128):
    model = OLS(df_sp_r.iloc[0+i:10+i,1], sm.add_constant(df_sp.iloc[0+i:10+i,4])['const'])
    result = model.fit(cov_type='HC0')
    oos=(result.params[0] - df_sp_r.iloc[10+i,1])**2
    pd_est_model_4_10=pd_est_model_4_10.append({'date':df_sp_r.iloc[10+i,0],'error_sq':oos},ignore_index = True)

In [289]:
# second model
pd_est_model2_4_10=pd.DataFrame(columns=['date','error_sq'])
for i in range(0,128):
    model = OLS(df_sp_r.iloc[0+i:10+i,1], sm.add_constant(df_sp.iloc[0+i:10+i,4]))
    result = model.fit(cov_type='HC0')
    oos=(result.params[0]+result.params[1]*df_sp.iloc[10+i,4] - df_sp_r.iloc[10+i,1])**2
    pd_est_model2_4_10=pd_est_model2_4_10.append({'date':df_sp_r.iloc[10+i,0],'error_sq':oos},ignore_index = True)

In [291]:
pd_est_model_4_10['error_sq'].sum()

4.42049854291729

In [292]:
pd_est_model2_4_10['error_sq'].sum()

5.376231554054691

In [293]:
#20 years

#first model 
pd_est_model_4_20=pd.DataFrame(columns=['date','error_sq'])
for i in range(0,118):
    model = OLS(df_sp_r.iloc[0+i:20+i,1], sm.add_constant(df_sp.iloc[0+i:20+i,4])['const'])
    result = model.fit(cov_type='HC0')
    oos=(result.params[0] - df_sp_r.iloc[20+i,1])**2
    pd_est_model_4_20=pd_est_model_4_20.append({'date':df_sp_r.iloc[20+i,0],'error_sq':oos},ignore_index = True)

In [294]:
# second model
pd_est_model2_4_20=pd.DataFrame(columns=['date','error_sq'])
for i in range(0,118):
    model = OLS(df_sp_r.iloc[0+i:20+i,1], sm.add_constant(df_sp.iloc[0+i:20+i,4]))
    result = model.fit(cov_type='HC0')
    oos=(result.params[0]+result.params[1]*df_sp.iloc[20+i,4] - df_sp_r.iloc[20+i,1])**2
    pd_est_model2_4_20=pd_est_model2_4_20.append({'date':df_sp_r.iloc[20+i,0],'error_sq':oos},ignore_index = True)

In [295]:
pd_est_model_4_20['error_sq'].sum()

4.132557567404263

In [297]:
pd_est_model2_4_20['error_sq'].sum()

4.23587269626587

In [298]:
#30 years 

#first model
pd_est_model_4_30=pd.DataFrame(columns=['date','error_sq'])
for i in range(0,108):
    model = OLS(df_sp_r.iloc[0+i:30+i,1], sm.add_constant(df_sp.iloc[0+i:30+i,4])['const'])
    result = model.fit(cov_type='HC0')
    oos=(result.params[0] - df_sp_r.iloc[30+i,1])**2
    pd_est_model_4_30=pd_est_model_4_30.append({'date':df_sp_r.iloc[30+i,0],'error_sq':oos},ignore_index = True)

In [299]:
#second model
pd_est_model2_4_30=pd.DataFrame(columns=['date','error_sq'])
for i in range(0,108):
    model = OLS(df_sp_r.iloc[0+i:30+i,1], sm.add_constant(df_sp.iloc[0+i:30+i,4]))
    result = model.fit(cov_type='HC0')
    oos=(result.params[0]+result.params[1]*df_sp.iloc[30+i,4] - df_sp_r.iloc[30+i,1])**2
    pd_est_model2_4_30=pd_est_model2_4_30.append({'date':df_sp_r.iloc[30+i,0],'error_sq':oos},ignore_index = True)

In [300]:
pd_est_model_4_30['error_sq'].sum()

3.652791903163243

In [301]:
pd_est_model2_4_30['error_sq'].sum()

3.6897753096203325

There is no much difference. The first model is still better for predicting. CAPE is also not good for predicting the return which is out of sample whatever the period is for testing the model. CAPE is similar to E/P but not so much for computing the ratio we have to adjust for inflation and compute the average eps during the past several months/years. In other words, in these ratios we use practically the same things eps and the price but using these numbers they do not explain altering in the future returns as the model shows to us. It is still better to use the past average returns than to add CAPE or E/P to the model.

#5
We may use t test for proving statistical difference between the models, especially we can compute the means of two models and than assume that there is no difference between them. It means that the means shoud be equal. H0 hypothesis is that the means are equal, the alternative hypothesis is that they are not. 

In [302]:
#5_1
statistic, pvalue = ttest_ind(a=pd_est_model['error_sq'],b=pd_est_model2['error_sq'])
statistic

-0.698491656684882

In [303]:
#5_2
statistic, pvalue = ttest_ind(a=pd_est_model_2['error_sq'],b=pd_est_model2_2['error_sq'])
statistic

0.0127750377941473

In [305]:
#5_3 10 years
statistic, pvalue = ttest_ind(a=pd_est_model_3_10['error_sq'],b=pd_est_model2_3_10['error_sq'])
statistic

-1.2886791474075663

In [306]:
#5_3 20 years
statistic, pvalue = ttest_ind(a=pd_est_model_3_20['error_sq'],b=pd_est_model2_3_20['error_sq'])
statistic

-0.42254502139420164

In [307]:
#5_3 30 years
statistic, pvalue = ttest_ind(a=pd_est_model_3_30['error_sq'],b=pd_est_model2_3_30['error_sq'])
statistic

-0.21691208793317573

In [308]:
#5_4 10 years 
statistic, pvalue = ttest_ind(a=pd_est_model_4_10['error_sq'],b=pd_est_model2_4_10['error_sq'])
statistic

-0.9552731897105783

In [309]:
#5_4 20 years
statistic, pvalue = ttest_ind(a=pd_est_model_4_20['error_sq'],b=pd_est_model2_4_20['error_sq'])
statistic

-0.13472775691171784

In [310]:
#5_4 30 years
statistic, pvalue = ttest_ind(a=pd_est_model_4_30['error_sq'],b=pd_est_model2_4_30['error_sq'])
statistic

-0.05147973207322748

We may see that in each case h0 hypothesis is not rejected, meaning that statistically both models in each case have equal means. Therefore, they do not differ.

In [311]:
#5 (3) and (4) 10 years 
statistic, pvalue = ttest_ind(a=pd_est_model2_3_10['error_sq'],b=pd_est_model2_4_10['error_sq'])
statistic

0.349092927510412

In [312]:
#5 (3) and (4) 20 years 
statistic, pvalue = ttest_ind(a=pd_est_model2_3_20['error_sq'],b=pd_est_model2_4_20['error_sq'])
statistic

0.2914787578239047

In [313]:
#5 (3) and (4) 30 years 
statistic, pvalue = ttest_ind(a=pd_est_model2_3_30['error_sq'],b=pd_est_model2_4_30['error_sq'])
statistic

0.17325891738526447

I compare the second model with the different regressors. The means of both models do not statistically differ because H0 hypothesis is not rejected. 